In [21]:
import tensorflow as tf
import numpy as np

In [4]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [5]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
print(sess.run(f))
sess.close()

42


In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    print(f.eval())

42


In [10]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    print(f.eval())

42


In [11]:
sess = tf.InteractiveSession()
init.run()
print(f.eval())
sess.close()

42


In [12]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [13]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

In [15]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [16]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [19]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
housing.data.shape

(20640, 8)

In [18]:
housing.feature_names

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

Using the normal equation:

In [28]:
housing_plus_offset = np.c_[np.ones((housing.data.shape[0], 1)), housing.data]
X = tf.constant(housing_plus_offset, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [41]:
from sklearn.preprocessing import StandardScaler
scaled_housing = StandardScaler().fit_transform(housing.data)
scaled_offset_housing = np.c_[
    np.ones((housing.data.shape[0], 1)), scaled_housing]

using the normal equation with scaled data:

In [42]:
tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[ 2.0685577 ]
 [ 0.8296193 ]
 [ 0.11875156]
 [-0.26552734]
 [ 0.3056965 ]
 [-0.00450308]
 [-0.03932629]
 [-0.8998864 ]
 [-0.87054193]]


Manual gradient decent:

In [46]:
n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/housing.data.shape[0] * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_theta = theta.eval()
print(best_theta)

Epoch 0, MSE 9.413900375366211
Epoch 1000, MSE 0.5294526219367981
Epoch 2000, MSE 0.5245167016983032
Epoch 3000, MSE 0.5243282318115234
Epoch 4000, MSE 0.5243212580680847
[[ 2.0685523 ]
 [ 0.8295648 ]
 [ 0.11878298]
 [-0.26534548]
 [ 0.30551198]
 [-0.00449096]
 [-0.03932773]
 [-0.8997085 ]
 [-0.8703533 ]]


Using autodiff for the gradients...

In [47]:
n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_theta = theta.eval()
print(best_theta)

Epoch 0, MSE 5.8281569480896
Epoch 1000, MSE 0.5374355316162109
Epoch 2000, MSE 0.5248290300369263
Epoch 3000, MSE 0.5243416428565979
Epoch 4000, MSE 0.5243220925331116
[[ 2.0685525 ]
 [ 0.82975656]
 [ 0.1188428 ]
 [-0.26566607]
 [ 0.30575782]
 [-0.00447162]
 [-0.03933699]
 [-0.8990903 ]
 [-0.86975586]]


Using the builtin gradient descent optimizer:

In [49]:
n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_theta = theta.eval()
print(best_theta)

Epoch 0, MSE 9.966286659240723
Epoch 1000, MSE 0.5459184050559998
Epoch 2000, MSE 0.5256495475769043
Epoch 3000, MSE 0.5244531631469727
Epoch 4000, MSE 0.5243389010429382
[[ 2.0685525 ]
 [ 0.8313976 ]
 [ 0.11915641]
 [-0.26875275]
 [ 0.30830505]
 [-0.00437884]
 [-0.03939844]
 [-0.8952657 ]
 [-0.8661216 ]]


Using a momentum optimizer:

In [53]:
n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_theta = theta.eval()
print(best_theta)

Epoch 0, MSE 6.115485668182373
Epoch 1000, MSE 0.5243209600448608
Epoch 2000, MSE 0.5243209600448608
Epoch 3000, MSE 0.5243209600448608
Epoch 4000, MSE 0.5243209600448608
[[ 2.0685577 ]
 [ 0.8296182 ]
 [ 0.11875145]
 [-0.265525  ]
 [ 0.3056947 ]
 [-0.00450307]
 [-0.03932623]
 [-0.899888  ]
 [-0.8705432 ]]


### Placeholders

In [52]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5
with tf.Session() as sess:
    B_1 = B.eval(feed_dict={ A: [[1, 2, 3]] })
    B_2 = B.eval(feed_dict={ A: [[4, 5, 6], [7, 8, 9]] })
    
print("{0}\n{1}".format(B_1, B_2))

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


### Minibatch training

In [71]:
num_epochs = 5000
learning_rate = 0.01
batch_size = 10000
num_batches = int(np.ceil(housing.data.shape[0] / batch_size))

tf.reset_default_graph()
X = tf.placeholder(shape=(None, housing.data.shape[1] + 1),
    dtype=tf.float32, name="X")
y= tf.placeholder(shape=(None, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

def fetch_batch(epoch, batch_index, batch_size):
    start = batch_index * batch_size
    end = np.min([(batch_index + 1) * batch_size,
                  scaled_offset_housing.shape[0]])
    return (scaled_offset_housing[start:end, :], 
            housing.target.reshape(-1, 1)[start:end,:])

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epochs):
        for batch_index in range(num_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, 0))#mse.eval()))
        
    best_theta = theta.eval()
print(best_theta)

Epoch 0, MSE 0
Epoch 1000, MSE 0
Epoch 2000, MSE 0
Epoch 3000, MSE 0
Epoch 4000, MSE 0
[[ 2.04163623e+00]
 [ 7.76685953e-01]
 [ 1.02068946e-01]
 [-1.75507620e-01]
 [ 2.32567772e-01]
 [-8.55466409e-04]
 [-4.33079973e-02]
 [-8.76474440e-01]
 [-8.66599262e-01]]


### Saving and restoring Models

In [74]:
n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.Saver() # create saver after all of the variables have been created

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
            saver.save(sess, "/tmp/tf.ckpt")
        sess.run(training_op)
        
    best_theta = theta.eval()
    saver.save(sess, "/tmp/tf_final.ckpt")
print(best_theta)

Epoch 0, MSE 9.707866668701172
Epoch 1000, MSE 0.5390802621841431
Epoch 2000, MSE 0.5266116857528687
Epoch 3000, MSE 0.5246871113777161
Epoch 4000, MSE 0.5243797898292542
[[ 2.0685525 ]
 [ 0.8330944 ]
 [ 0.11941039]
 [-0.27207884]
 [ 0.31110823]
 [-0.00430847]
 [-0.0394561 ]
 [-0.89182055]
 [-0.86287963]]


In [76]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/tf_final.ckpt")
    print(theta.eval())

INFO:tensorflow:Restoring parameters from /tmp/tf_final.ckpt
[[ 2.0685525 ]
 [ 0.8330944 ]
 [ 0.11941039]
 [-0.27207884]
 [ 0.31110823]
 [-0.00430847]
 [-0.0394561 ]
 [-0.89182055]
 [-0.86287963]]


## TensorBoard

First, we need to log the error rate

In [78]:
from datetime import datetime

logdir = "/tmp/tf/run-{}".format(datetime.utcnow().isoformat())

n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            summary_str = mse_summary.eval()
            file_writer.add_summary(summary_str, epoch)
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_theta = theta.eval()
    
file_writer.close()

Epoch 0, MSE 11.783705711364746
Epoch 1000, MSE 0.5262514352798462
Epoch 2000, MSE 0.5244504809379578
Epoch 3000, MSE 0.5243350267410278
Epoch 4000, MSE 0.5243229269981384


Let's add some scope to the error and MSE

In [79]:
from datetime import datetime

logdir = "/tmp/tf/run-{}".format(datetime.utcnow().isoformat())

n_epochs = 5000
learning_rate = 0.01

tf.reset_default_graph()
X = tf.constant(scaled_offset_housing, dtype=tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(
    tf.random_uniform([housing.data.shape[1] + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="y_pred")

with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            summary_str = mse_summary.eval()
            file_writer.add_summary(summary_str, epoch)
        if epoch % 1000 == 0:
            print("Epoch {0}, MSE {1}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_theta = theta.eval()
    
file_writer.close()

Epoch 0, MSE 10.312064170837402
Epoch 1000, MSE 0.5264008045196533
Epoch 2000, MSE 0.5245612263679504
Epoch 3000, MSE 0.5243558287620544
Epoch 4000, MSE 0.5243263840675354
